# Análise de Sentimentos


### Carregando o dataset a ser utilizado

In [1]:
import pandas as pd
from os import system
import re
import numpy as np
import sys
print(sys.executable)

# imdb_train = pd.read_csv('datasets/csv/imdb_train.csv')
# imdb_test = pd.read_csv('datasets/csv/imdb_test.csv')

iac_train = pd.read_csv('datasets/csv/iac_convinceme_train.csv', sep= ";")
iac_test = pd.read_csv('datasets/csv/iac_create_debate_test.csv', sep= ";")

/Users/matheussilva/anaconda3/envs/p3workshop/bin/python


### Text Vectorization

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # used for saving and loading sklearn objects
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices

# system("mkdir datasets/data_preprocessors")
# system("mkdir datasets/vectorized_data")

# Unigram Counts

unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
# unigram_vectorizer.fit(imdb_train['text'].values)
unigram_vectorizer.fit(iac_train['text'].values)

dump(unigram_vectorizer, 'datasets/data_preprocessors/unigram_vectorizer.joblib')

# unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')

# X_train_unigram = unigram_vectorizer.transform(imdb_train['text'].values)
X_train_unigram = unigram_vectorizer.transform(iac_train['text'].values)

# Save new model
# save_npz('datasets/vectorized_data/X_train_unigram.npz', X_train_unigram)

# Load created model
X_train_unigram = load_npz('datasets/vectorized_data/X_train_unigram.npz')


# Unigram Tf-Idf

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

dump(unigram_tf_idf_transformer, 'datasets/data_preprocessors/unigram_tf_idf_transformer.joblib')

# unigram_tf_idf_transformer = load('datasets/data_preprocessors/unigram_tf_idf_transformer.joblib')

X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

# Save new model
# save_npz('datasets/vectorized_data/X_train_unigram_tf_idf.npz', X_train_unigram_tf_idf)

# Load created model
X_train_unigram_tf_idf = load_npz('datasets/vectorized_data/X_train_unigram_tf_idf.npz')

#-----------------------------------

# Bigram Counts

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
# bigram_vectorizer.fit(imdb_train['text'].values)
bigram_vectorizer.fit(iac_train['text'].values)

dump(bigram_vectorizer, 'datasets/data_preprocessors/bigram_vectorizer.joblib')

# bigram_vectorizer = load('data_preprocessors/bigram_vectorizer.joblib')

# X_train_bigram = bigram_vectorizer.transform(imdb_train['text'].values)
X_train_bigram = bigram_vectorizer.transform(iac_train['text'].values)

# Save new model
# save_npz('datasets/vectorized_data/X_train_bigram.npz', X_train_bigram)

# Load created model
X_train_bigram = load_npz('datasets/vectorized_data/X_train_bigram.npz')


# Bigram Tf-Idf

bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

dump(bigram_tf_idf_transformer, 'datasets/data_preprocessors/bigram_tf_idf_transformer.joblib')

# bigram_tf_idf_transformer = load('datasets/data_preprocessors/bigram_tf_idf_transformer.joblib')

X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

# Save new model
# save_npz('datasets/vectorized_data/X_train_bigram_tf_idf.npz', X_train_bigram_tf_idf)

# Load created model
X_train_bigram_tf_idf = load_npz('datasets/vectorized_data/X_train_bigram_tf_idf.npz')

## Escolha da melhor solução de Text vectorization

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np

def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'{title}\nTrain score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

# y_train = imdb_train['label'].values
y_train = iac_train['sentiment'].values

train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 0.77 ; Validation score: 0.72

Unigram Tf-Idf
Train score: 0.71 ; Validation score: 0.68

Bigram Counts
Train score: 0.91 ; Validation score: 0.73

Bigram Tf-Idf
Train score: 0.73 ; Validation score: 0.67



## Escolha de melhores parâmetros para a execução

O melhor método de vetorização de texto para a base IAC, pelos parametros acima, é o Bigram Counts. Com uma Validação de 0.73

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

#IMDB
# X_train = X_train_bigram_tf_idf

#IAC
X_train = X_train_bigram

# Phase 1: loss, learning rate and initial learning rate

clf = SGDClassifier()

distributions = dict(
    loss=['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    learning_rate=['optimal', 'invscaling', 'adaptive'],
    eta0=uniform(loc=1e-7, scale=1e-2)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=50,
    dual=False
)
random_search_cv.fit(X_train, y_train)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

In [5]:
# Phase 2: penalty and alpha

clf = SGDClassifier()

distributions = dict(
    penalty=['l1', 'l2', 'elasticnet'],
    alpha=uniform(loc=1e-6, scale=1e-4)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=50
)
random_search_cv.fit(X_train, y_train)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

Best params: {'alpha': 6.0443045621198073e-05, 'penalty': 'elasticnet'}
Best score: 0.7369077166254782


In [6]:
# Salvando o melhor classificador

# Criar um novo classificador

# sgd_classifier = random_search_cv.best_estimator_
# dump(random_search_cv.best_estimator_, 'datasets/classifiers/sgd_classifier.joblib')

# Carregar classificador
sgd_classifier = load('datasets/classifiers/sgd_classifier.joblib')

/Users/matheussilva/anaconda3/envs/p3workshop/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


### Base IAC

In [7]:
X_test = bigram_vectorizer.transform(iac_test['text'].values)
y_test = iac_test['sentiment'].values

score = sgd_classifier.score(X_test, y_test)
print(score)

0.743890629450299


Precisão atingida com a Base IAC - focada em discussões

74%

Classificações: 

* positive
* neutral
* negative

### Classificando a base Reddit e exportar resultados

In [8]:
reddit_base = pd.read_csv("reddit_post_extraction.csv")

X_reddit = bigram_vectorizer.transform(reddit_base['content'].values)

resultado_classificacao = sgd_classifier.predict(X_reddit)
resultado_classificacao

array(['negative', 'neutral', 'negative', ..., 'positive', 'negative',
       'negative'], dtype='<U8')

In [9]:
csv_output = reddit_base.copy()
del csv_output['position']
csv_output.insert(6, "position",resultado_classificacao, True)

In [10]:
csv_output.to_csv("resultado_classificacao.csv", sep=";", index=False)

## Base de Dados - IMDB (Comparação)

Precisão encontrada na base de testes é de 90%. Usando como base de treinamento críticas de filmes do site Imdb, e as marcações já presentes na base extraida.

O processo de execução com uma base extraida para análise, sem quaisquer marcações é feito abaixo

In [11]:
X_test = bigram_vectorizer.transform(imdb_test['text'].values)
X_test = bigram_tf_idf_transformer.transform(X_test)
y_test = imdb_test['label'].values

score = sgd_classifier.score(X_test, y_test)
print(score)

NameError: name 'imdb_test' is not defined

Para cada uma das linhas no arquivo de testes, há uma célula respectiva no array que a classifica

Uma vez obtido o resultado, é possível salvar o resultado obtido em um .csv e usar o notebook CsvToVisu para formatar a saída para o formato aceito pela ferramenta Visu.

In [ ]:
output = sgd_classifier.predict(X_test)
print(output)

# Métricas de Resultados

Base de Treino: "IAC_ConvinceMe_Train.csv"
   * Positive: 52209
   * Neutral: 53513
   * Negative: 53169
   

Base de Testes: "IAC_CreateDebate_test.csv"
   * Postive: 1601
   * Neutral: 5608
   * Negative: 10346

### Matriz de Confusão

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.metrics import plot_confusion_matrix

* <h5> Usando base de teste create_debate </h5>

In [ ]:
### Base de teste: iac_create_debate_test.csv

create_debate = pd.read_csv('datasets/csv/iac_create_debate_test.csv', sep= ";")

X = bigram_vectorizer.transform(create_debate['text'].values)
Y = create_debate['sentiment'].values

figura = plt.figure(figsize=(15,15))
plot_confusion_matrix(sgd_classifier, X, Y, cmap=plt.cm.Greens, display_labels=['oposição', 'neutro', 'apoio'])
plt.savefig('Matriz_Confusao_CreateDebate.png')
plt.show()

   * <h5> Usando base de teste com sample aleatório (12% da base o ConvinceMe)</h5>

In [ ]:
convince_me = pd.read_csv('datasets/csv/iac_convinceme_original_full.csv', sep= ";").sample(frac=.12)

X = bigram_vectorizer.transform(convince_me['text'].values)
Y = convince_me['sentiment'].values

figura = plt.figure(figsize=(15,15))
plot_confusion_matrix(sgd_classifier, X, Y, cmap=plt.cm.Greens, display_labels=['oposição', 'neutro', 'apoio'])
plt.savefig('Matriz_Confusao_ConvinceMe10Percent.png')
plt.show()